<a href="https://colab.research.google.com/github/aaagairi/Aditi_Gairi_CSEAIML-B_SmartFarming/blob/main/Weather_forecast_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries
import requests
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from datetime import datetime, timedelta

In [ ]:
#location
def get_coordinates(city):
    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    response = requests.get(geo_url)
    data = response.json()

    if "results" in data and data["results"]:
        latitude = data["results"][0]["latitude"]
        longitude = data["results"][0]["longitude"]
        return latitude, longitude
    else:
        raise ValueError("City not found. Please try another city.")

In [ ]:
#api
def fetch_weather_forecast(lat, lon):
    api_url = (
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}&daily=temperature_2m_max,"
        f"precipitation_sum,windspeed_10m_max&forecast_days=7&timezone=Asia/Kolkata"
    )
    response = requests.get(api_url)
    weather = response.json()["daily"]

    df = pd.DataFrame({
        "date": weather["time"],
        "temperature": weather["temperature_2m_max"],
        "precipitation": weather["precipitation_sum"],
        "wind_speed": weather["windspeed_10m_max"]
    })
    return df

In [ ]:
#alerts
def generate_alerts(row):
    alerts = []
    if row["predicted_temperature"] > 35:
        alerts.append("High temperature – risk of crop heat stress")
    if row["precipitation"] > 15:
        alerts.append("Heavy rain – possible flooding or waterlogging")
    if row["wind_speed"] > 30:
        alerts.append("High wind – risk of crop damage")
    return alerts

In [ ]:
#user input & prediction
city = input("Enter your city or village name: ")
try:
    latitude, longitude = get_coordinates(city)
    df = fetch_weather_forecast(latitude, longitude)

    X = df[["precipitation", "wind_speed"]]
    y = df["temperature"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = SVR()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print("\nModel Performance:")
    print("R2 Score:", r2_score(y_test, y_pred))
    print("RMSE:", rmse)

    df["predicted_temperature"] = model.predict(df[["precipitation", "wind_speed"]])
    df["alerts"] = df.apply(generate_alerts, axis=1)

    print("\n7-Day Farming Forecast:")
    for _, row in df.iterrows():
        print(f"\nDate: {row['date']}")
        print(f"Predicted Temp: {row['predicted_temperature']:.2f}°C")
        print(f"Rainfall: {row['precipitation']} mm")
        print(f"Wind Speed: {row['wind_speed']} km/h")
        if row['alerts']:
            print("⚠️ Alerts:")
            for alert in row['alerts']:
                print(" -", alert)
        else:
            print("✅ No farming alerts")

except Exception as e:
    print("Error:", e)


Enter your city or village name: Delhi

Model Performance:
R2 Score: -1.88761927763387
RMSE: 2.0781229638774192

7-Day Farming Forecast:

Date: 2025-04-30
Predicted Temp: 33.45°C
Rainfall: 0.0 mm
Wind Speed: 11.9 km/h
✅ No farming alerts

Date: 2025-05-01
Predicted Temp: 33.41°C
Rainfall: 0.0 mm
Wind Speed: 16.6 km/h
✅ No farming alerts

Date: 2025-05-02
Predicted Temp: 33.60°C
Rainfall: 1.1 mm
Wind Speed: 13.7 km/h
✅ No farming alerts

Date: 2025-05-03
Predicted Temp: 32.25°C
Rainfall: 8.2 mm
Wind Speed: 11.8 km/h
✅ No farming alerts

Date: 2025-05-04
Predicted Temp: 32.10°C
Rainfall: 9.6 mm
Wind Speed: 10.8 km/h
✅ No farming alerts

Date: 2025-05-05
Predicted Temp: 32.09°C
Rainfall: 10.5 mm
Wind Speed: 9.8 km/h
✅ No farming alerts

Date: 2025-05-06
Predicted Temp: 32.16°C
Rainfall: 6.6 mm
Wind Speed: 10.3 km/h
✅ No farming alerts
